In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def replace_nan_values(df):
    df.replace(['?', 'NA', 'N/A', 'null', 'None', ' '], np.nan, inplace=True)
    return df

def check_nulls(dataset):
    null = dataset.isnull().sum()
    ratio = (null/dataset.shape[0])*100
    return pd.DataFrame({'Null_Sum': null, 'Ratio': ratio}).T


In [5]:
import os
save_path = '../data/preprocessed'
os.makedirs(save_path, exist_ok= True) 

In [6]:
dataset = pd.read_csv('../data/original/adult.csv')

In [7]:
dataset = replace_nan_values(dataset)
print(check_nulls(dataset))

          age    workclass  fnlwgt  education  educational-num  \
Null_Sum  0.0  2799.000000     0.0        0.0              0.0   
Ratio     0.0     5.730724     0.0        0.0              0.0   

          marital-status   occupation  relationship  race  gender  \
Null_Sum             0.0  2809.000000           0.0   0.0     0.0   
Ratio                0.0     5.751198           0.0   0.0     0.0   

          capital-gain  capital-loss  hours-per-week  native-country  income  
Null_Sum           0.0           0.0             0.0      857.000000     0.0  
Ratio              0.0           0.0             0.0        1.754637     0.0  


In [8]:
# Check nulls
check_nulls(dataset)
# Define columns with numerical (regression) NaNs and categorical (class) NaNs
class_columns = ['workclass', 'occupation', 'native-country']
dataset[class_columns] = dataset[class_columns].fillna(dataset[class_columns].mode().iloc[0])

# Verify
print(dataset.isnull().sum())

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64


In [9]:
categorical_data = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country'] #education pense pas onehotencode
label_data = ['gender']


In [10]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [13]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_data)], remainder='passthrough')
X = pd.DataFrame(X)
X= ct.fit_transform(X)
feature_names = ct.get_feature_names_out()


ValueError: A given column is not a column of the dataframe

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
ct_gender = ColumnTransformer(transformers=[('encoder', LabelEncoder(), label_data)], remainder='passthrough')
y = le_y.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False) 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
feature_names[-1]

NameError: name 'feature_names' is not defined

In [ ]:
X_train = pd.DataFrame(X_train, columns=feature_names[:-1])
X_test = pd.DataFrame(X_test, columns=feature_names[:-1])
y_train = pd.DataFrame(y_train, columns=feature_names[-1])
y_test = pd.DataFrame(y_test, columns=feature_names[-1])


,encoder__workclass_Federal-gov,encoder__workclass_Local-gov,encoder__workclass_Never-worked,encoder__workclass_Private,encoder__workclass_Self-emp-inc,encoder__workclass_Self-emp-not-inc,encoder__workclass_State-gov,encoder__workclass_Without-pay,encoder__education_10th,encoder__education_11th,...,encoder__native-country_United-States,encoder__native-country_Vietnam,encoder__native-country_Yugoslavia,remainder__age,remainder__fnlwgt,remainder__educational-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week,remainder__income
0,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,5.290867,...,3.58519,0.0,0.0,1.823437,2.147686,2.722733,0.0,0.0,3.228067,0
1,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,2.771625,0.850488,3.500656,0.0,0.0,4.035084,0
2,0.0,4.079611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,2.04225,3.190735,4.667542,0.0,0.0,3.228067,1
3,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,3.20925,1.518168,3.889618,1.031677,0.0,3.228067,1
4,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,1.312875,0.980058,3.889618,0.0,0.0,2.42105,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,1.969312,2.436504,4.667542,0.0,0.0,3.066663,0
48838,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,2.9175,1.461834,3.500656,0.0,0.0,3.228067,1
48839,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,4.230375,1.438502,3.500656,0.0,0.0,3.228067,0
48840,0.0,0.0,0.0,2.314063,0.0,0.0,0.0,0.0,0.0,0.0,...,3.58519,0.0,0.0,1.604625,1.907996,3.500656,0.0,0.0,1.614033,0


In [ ]:
# dataset.to_csv(f'{save_path}/adult.csv')
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
df_train.to_csv(f'{save_path}/adult/train.csv', index=False)
df_test.to_csv(f'{save_path}/adult/test.csv', index=False)

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
# from sklearn.metrics import ConfusionMatrixDisplay

# # Predict
# y_pred = model.predict(X_test)

# # Confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(cm)
# disp.plot(cmap='Blues')
# plt.title("SVM Confusion Matrix")
# plt.show()
